**# Model Training using AlexNet Architecture#**

In [1]:
import os
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.utils import Sequence, to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
#paths and parameters
TRAIN_DIR = '/workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/preprocessed_data/train'
VAL_DIR = '/workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/preprocessed_data/validation'
MODEL_PATH = '/workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/model/modeldp1_best.h5'
INPUT_SHAPE = (128, 128, 1)
EPOCHS = 20
BATCH_SIZE = 128
CLASS_NAMES = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
CLASS_TO_IDX = {name: idx for idx, name in enumerate(CLASS_NAMES)}

In [3]:
class NPYDataGenerator(Sequence):
    def __init__(self, root_dir, class_names, class_to_idx, batch_size=32, shuffle=True):
        self.root_dir = root_dir
        self.class_names = class_names
        self.class_to_idx = class_to_idx
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.samples = []
        for class_name in class_names:
            class_dir = os.path.join(root_dir, class_name)
            if not os.path.isdir(class_dir):
                continue
            for fname in os.listdir(class_dir):
                if fname.endswith('.npy'):
                    self.samples.append((os.path.join(class_dir, fname), class_to_idx[class_name]))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.samples) / self.batch_size))

    def __getitem__(self, idx):
        batch_samples = self.samples[idx * self.batch_size:(idx + 1) * self.batch_size]
        X = []
        y = []
        for fpath, label in batch_samples:
            img = np.load(fpath)
            X.append(img)
            y.append(label)
        X = np.stack(X)
        y = np.array(y)
        y = to_categorical(y, num_classes=len(self.class_names))
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.samples)

In [4]:
#Build AlexNet model
def build_my_model(input_shape=(128, 128, 1), num_classes=7):
    model = Sequential()
    model.add(Conv2D(48, (11, 11), strides=4, padding='same', input_shape=input_shape))  # 48 instead of 96
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

    model.add(Conv2D(128, (5, 5), padding='same'))  # 128 instead of 256
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

    model.add(Conv2D(192, (3, 3), padding='same', activation='relu'))  # 192 instead of 384
    model.add(Conv2D(192, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))  # 128 instead of 256
    model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))  # 512 instead of 4096
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model


In [5]:
# Create generators for training and validation
train_gen = NPYDataGenerator(TRAIN_DIR, CLASS_NAMES, CLASS_TO_IDX, batch_size=BATCH_SIZE)
val_gen = NPYDataGenerator(VAL_DIR, CLASS_NAMES, CLASS_TO_IDX, batch_size=BATCH_SIZE, shuffle=False)

print(f"Number of training batches: {len(train_gen)}")
print(f"Number of validation batches: {len(val_gen)}")

Number of training batches: 226
Number of validation batches: 56


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
#Training will stop when the validation accuracy is not improving for 3 consecutive epochs.

In [ ]:
model = build_my_model(input_shape=INPUT_SHAPE, num_classes=len(CLASS_NAMES))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint = ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1)

model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stop]
)

Number of training batches: 226
Number of validation batches: 56
Epoch 1/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2265 - loss: 1.8694
Epoch 1: val_accuracy improved from -inf to 0.30201, saving model to /workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/model/modeldp1_best.h5
WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
226/226 ━━━━━━━━━━━━━━━━━━━━ 395s 2s/step - accuracy: 0.2266 - loss: 1.8691 - val_accuracy: 0.3020 - val_loss: 1.7605
Epoch 2/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3765 - loss: 1.5801
Epoch 2: val_accuracy improved from 0.30201 to 0.37334, saving model to /workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/model/modeldp1_best.h5
WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
226/226 ━━━━━━━━━━━━━━━━━━━━ 460s 2s/step - accuracy: 0.3765 - loss: 1.5799 - val_accuracy: 0.3733 - val_loss: 1.6277
Epoch 3/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4541 - loss: 1.4069
Epoch 3: val_accuracy improved from 0.37334 to 0.47141, saving model to /workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/model/modeldp1_best.h5
WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
226/226 ━━━━━━━━━━━━━━━━━━━━ 457s 2s/step - accuracy: 0.4542 - loss: 1.4068 - val_accuracy: 0.4714 - val_loss: 1.3612
Epoch 4/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4951 - loss: 1.3016
Epoch 4: val_accuracy improved from 0.47141 to 0.49264, saving model to /workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/model/modeldp1_best.h5
WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
226/226 ━━━━━━━━━━━━━━━━━━━━ 445s 2s/step - accuracy: 0.4951 - loss: 1.3016 - val_accuracy: 0.4926 - val_loss: 1.3117
Epoch 5/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5300 - loss: 1.2312
Epoch 5: val_accuracy did not improve from 0.49264
226/226 ━━━━━━━━━━━━━━━━━━━━ 460s 2s/step - accuracy: 0.5299 - loss: 1.2312 - val_accuracy: 0.4558 - val_loss: 1.3961
Epoch 6/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5391 - loss: 1.2059
Epoch 6: val_accuracy improved from 0.49264 to 0.51670, saving model to /workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/model/modeldp1_best.h5
WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
226/226 ━━━━━━━━━━━━━━━━━━━━ 476s 2s/step - accuracy: 0.5392 - loss: 1.2058 - val_accuracy: 0.5167 - val_loss: 1.2661
Epoch 7/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5589 - loss: 1.1576
Epoch 7: val_accuracy improved from 0.51670 to 0.53566, saving model to /workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/model/modeldp1_best.h5
WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
226/226 ━━━━━━━━━━━━━━━━━━━━ 449s 2s/step - accuracy: 0.5589 - loss: 1.1576 - val_accuracy: 0.5357 - val_loss: 1.2231
Epoch 8/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5811 - loss: 1.0993
Epoch 8: val_accuracy did not improve from 0.53566
226/226 ━━━━━━━━━━━━━━━━━━━━ 350s 2s/step - accuracy: 0.5811 - loss: 1.0993 - val_accuracy: 0.5159 - val_loss: 1.2564
Epoch 9/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6023 - loss: 1.0534
Epoch 9: val_accuracy improved from 0.53566 to 0.53948, saving model to /workspaces/Emotion_detector/notebooks/FC110533_Dulith/base_model/model/modeldp1_best.h5
WARNING:absl:You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 
226/226 ━━━━━━━━━━━━━━━━━━━━ 413s 2s/step - accuracy: 0.6022 - loss: 1.0534 - val_accuracy: 0.5395 - val_loss: 1.2342
Epoch 10/20
226/226 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6138 - loss: 1.0113
Epoch 10: val_accuracy did not improve from 0.53948
226/226 ━━━━━━━━━━━━━━━━━━━━ 371s 2s/step - accuracy: 0.6138 - loss: 1.0114 - val_accuracy: 0.5137 - val_loss: 1.3069
<keras.src.callbacks.history.History at 0xffff02a656a0>